# fhirclient 

**Note: this example uses FHIR DSTU3 whereas synthea now supports FHIR R4 so you'll need a different version of the fhirclient library to deal with each dataset**

The _fhirclient_, a flexible Python client for FHIR servers supporting the SMART on FHIR protocol.

_fhirclient_ versioning is not identical to FHIR versioning, see the full table for reference.

| Version | FHIR | |
| --- | --- | --- |
| **4.0.0** | `4.0.0` | (R4) |
| **3.0.0** | `3.0.0` | (STU-3) |
| **1.0.3** | `1.0.2` | (DSTU 2) |
| **1.0** | `1.0.1` | (DSTU 2) |

## Installation

For FHIR version 3

```bash
pip install fhirclient
```

or for FHIR version 4

```bash
pip install git+https://github.com/smart-on-fhir/client-py.git
```

**NOTE: We'll use FHIR R4 here so you need to install from GitHub**

## Documentation

Technical documentation is available at [docs](https://docs.smarthealthit.org/client-py/).

## Client Use

To connect to a SMART on FHIR server (or any open FHIR server), you can use the `FHIRClient` class.
It will initialize and handle a `FHIRServer` instance, your actual handle to the FHIR server you'd like to access.

### Read Data from Server

To read a given patient from an open FHIR server, you can use:

In [1]:
from pprint import pprint
from fhirclient import client

settings = {
'app_id': 'my_web_app',
'api_base': 'http://hapi.fhir.org/baseR4'
}

smart = client.FHIRClient(settings=settings)

import fhirclient.models.patient as p

patient = p.Patient.read('2543713', smart.server)
print(patient.birthDate.isostring)
print(smart.human_name(patient.name[0]))

1974-12-25
Peter James Chalmers


If this is a protected server, you will first have to send your user to the authorize endpoint to log in.

1. Call `smart.authorize_url` to obtain the correct URL.
    2. You can use `smart.prepare()`, which will return `False` if the server is protected and you need to authorize.
    2. The `smart.ready` property has the same purpose, it will however not retrieve the server's _CapabilityStatement_ resource and hence is only useful as a quick check whether the server instance is ready.

In [2]:
smart = client.FHIRClient(settings=settings)
smart.ready

# prints False
print(smart.prepare())

# prints True after fetching CapabilityStatement
print(smart.ready)

# prints True
print(smart.prepare())

# prints True immediately
print(smart.authorize_url)

True
True
True
None


You can work with the `FHIRServer` class directly, without using `FHIRClient`, but this is not recommended:

### Search Records on Server

You can also search for resources matching a particular set of criteria:

In [3]:
smart = client.FHIRClient(settings=settings)

import fhirclient.models.patient as p

search = p.Patient.where(struct={'family': 'Cushing'})
patients = search.perform_resources(smart.server)
# print(observations)

pprint(patients[0].as_json())

{'id': '2581341',
 'identifier': [{'type': {'coding': [{'code': 'MR',
                                      'system': 'http://hl7.org/fhir/v2/0203'}]},
                 'value': 'dbee78a3-a72d-41a2-b705-0023498f5228'}],
 'meta': {'lastUpdated': '2021-10-07T05:30:35.359+00:00',
          'source': '#ghjs1LZGhTvv0mp2',
          'versionId': '1'},
 'name': [{'family': 'Cushing', 'given': ['Caleb']}],
 'resourceType': 'Patient',
 'text': {'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div '
                 'class="hapiHeaderText">Caleb <b>CUSHING </b></div><table '
                 'class="hapiPropertyTable"><tbody><tr><td>Identifier</td><td>dbee78a3-a72d-41a2-b705-0023498f5228</td></tr></tbody></table></div>',
          'status': 'generated'}}


In [4]:
# to get the raw Bundle instead of resources only, you can use:
bundle = search.perform(smart.server)
pprint(bundle.as_json())

{'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/Patient/2581341',
            'resource': {'id': '2581341',
                         'identifier': [{'type': {'coding': [{'code': 'MR',
                                                              'system': 'http://hl7.org/fhir/v2/0203'}]},
                                         'value': 'dbee78a3-a72d-41a2-b705-0023498f5228'}],
                         'meta': {'lastUpdated': '2021-10-07T05:30:35.359+00:00',
                                  'source': '#ghjs1LZGhTvv0mp2',
                                  'versionId': '1'},
                         'name': [{'family': 'Cushing', 'given': ['Caleb']}],
                         'resourceType': 'Patient',
                         'text': {'div': '<div '
                                         'xmlns="http://www.w3.org/1999/xhtml"><div '
                                         'class="hapiHeaderText">Caleb '
                                         '<b>CUSHING </b></div><table '
    

## Data Model Use

The client contains data model classes, built using ```fhir-parser```, that handle (de)serialization and allow to work with FHIR data in a Pythonic way.

### Initialize Data Model

In [5]:
import fhirclient.models.patient
import fhirclient.models.humanname

data = {'id': 'patient-1'}
patient = fhirclient.models.patient.Patient(data)

print(patient.id)

patient-1


In [6]:
name = fhirclient.models.humanname.HumanName()
name.given = ['Peter']
name.family = 'Parker'

patient.name = [name]
pprint(patient.as_json())

{'id': 'patient-1',
 'name': [{'family': 'Parker', 'given': ['Peter']}],
 'resourceType': 'Patient'}


In [7]:
name.given = 'Peter'
patient.as_json() # throws FHIRValidationError: because we incorrectly set the name to a string

FHIRValidationError: {root}:
  name.0:
    given:
      Expecting property "given" on <class 'fhirclient.models.humanname.HumanName'> to be list, but is <class 'str'>

### Initialize from JSON file

In [8]:
import json
import fhirclient.models.patient

with open('patient.json', 'r') as h:
    pjs = json.load(h)
    patient = fhirclient.models.patient.Patient(pjs)
    print(patient.name[0].family)
    print(patient.name[0].given[0])
    print(patient.gender)
    print(patient.birthDate.isostring)

Sky
Luc
male
1980-01-01
